# Capstone Project - Final Assignment
## If you have time to visit one city in the Netherlands?

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>

1. <a href="#section1">Introduction</a>
2. <a href="#section2">Objective</a>
3. <a href="#section3">Data</a>

</font>
</div>

<a id="section1"></a>
## 1. Introduction

The Netherlands is a great country to visit for a short break or a longer holiday. And there are many nice cities known for their restaurants, nightlife, museums, market, squares and parks. So deciding which Netherlands city to visit can be difficult. Especially, if you have time to visit one city. What city would you visit?

The aim of this project is to collect data and provide a recommendation to people who wants to visit the Netherlands for a short stay. The recommendation are a help to make a decision which city to visit in the Netherlands. They can choose a city by venue type, tips of venues and popular places.

According to the website https://www.holland.com/global/tourism.htm the most popular destinations in the Netherlands are:

| City      | LAT | LONG
| ----------| ----------| ----------|
Amsterdam	
Leeuwarden	
Arnhem	
Gouda
Rotterdam	
Leiden	
Breda	
Groningen
The Hague	
Haarlem	
Delft	
Utrecht	
Volendam	
Den Bosch	
Maastricht	
Alkmaar	
Eindhoven	
Nijmegen	
Roermond	
Scheveningen



<a id="section2"></a>
## 2. Objective

The objective of this project is to compare the above 20 cities and determine how similar or dissimilar they are. The aim is to determine:

1. The similarity or dissimilarity of the 20 cities in the Netherlands based on tourism venues like; events, food, outdoors & recreation, shop & service, and travel & transport:  
        a. What are the most common venues types (venues count)  
        b. What are the recommending scores of the top 10 venues based on ‘ratings’.  
    
2. Clusters these 20 cities performing a K-means clustering. 

<a id="section3"></a>
## 3. Data

For this project the data of foursquare (website: www.foursquare.com) will be used for information about venues, tips and check-ins. The Foursquare explore function will be used to get the most common venue categories in each city, and then use this feature to group the cities into clusters. Another venue query will be performed to retrieve venue ratings for each city to find the most popular places. Note that there is max query limit of 50 ratings in FourSquare API for free subscription.